# Crawl Popular-time from google

## 1. 爬取景點基本資訊與熱門時段

`注意: google 爬蟲可能因為 google 網頁改版而失效`

In [1]:
from sub_project.google_place_crawler import main_controller

In [3]:
Crawler = main_controller.Crawler(
        file_path = 'data/raw/scenic_spot/spot_list(manual).xlsx',
        sheet_name = 'spot_use',
        target = 'name',
        output_folder = 'data/proccess/1_popular_time',
        chunk_size = 8
    )
Crawler.run()

Checking output folder!
No data in output folder!
target: 33
Current progress: 0/4
Current progress: 1/4
Current progress: 2/4
Current progress: 3/4
Current progress: 4/4


## 2. 合併資料

In [9]:
import pandas as pd

df = pd.read_json('data/proccess/1_popular_time/main.json', lines=True, orient='records')
main_df = pd.read_excel('data/raw/scenic_spot/spot_list(manual).xlsx', sheet_name='spot_use', engine='openpyxl')
main_df.merge(df, how='left', left_on='name', right_on='keyword').to_pickle('data/proccess/1_popular_time/main.pkl')


## 3. 整理熱門時間資料

In [11]:
import pandas as pd

df = pd.read_pickle('data/proccess/1_popular_time/main.pkl')

timeflow_all_df = pd.DataFrame()
for i in range(df.shape[0]):
    if df.loc[i, 'have_time_flow'] == False:
        pass
    else:
        timeflow = df.loc[i, 'time_flow']
        for w in timeflow.keys():
            for time in timeflow[w].keys():
                if time.find('目前繁忙程度')!=-1:
                    time_miss = list(set(['{}時'.format(i) for i in range(24)][-len(timeflow[w]):]) - set(timeflow[w].keys()))[0]
                    timeflow[w][time_miss] = timeflow[w][time].replace(')','')
                    del timeflow[w][time]

        timeflow_df = pd.DataFrame(timeflow).rename(
            columns={'0':'日', '1':'一', '2':'二', '3':'三', '4':'四', '5':'五', '6':'六'}
                ).T
        timeflow_df = timeflow_df.reset_index().rename(columns={'index':'week'})
        timeflow_df['keyword'] = df.loc[i, 'keyword']
        timeflow_df['遊憩據點'] = df.loc[i, '遊憩據點']
        timeflow_df['result_name'] = df.loc[i, 'result_name']

        timeflow_all_df = timeflow_all_df.append(timeflow_df)
col = ['遊憩據點', 'result_name', 'week'] + ['{}時'.format(i) for i in range(24)]

timeflow_all_df[col].to_csv('data/final/1_popular_time.csv', index=False)
timeflow_all_df[col].to_csv('../data/popular_time.csv', index=False)